## 學習率 (learning rate)
用於解決類似 MBGD 出現的缺點問題\
[參考資料](https://www.796t.com/content/1545433422.html)

### Momentum
SGD 在 ravines 的情況下容易被困住， ravines 就是曲面的一個方向比另一個方向更陡，這時 SGD 會發生震盪而遲遲不能接近極小值：
*   梯度更新規則:\
    Momentum 通過加入 $\gamma v_{t-1}$ ，可以加速 SGD， 並且抑制震盪\
    $v_t = \gamma v_{t-1} + \eta \nabla_\theta J(\theta)$\
    $\theta = \theta - v_t$\
    當我們將一個小球從山上滾下來時，沒有阻力的話，它的動量會越來越大，但是如果遇到了阻力，速度就會變小。\
    加入的這一項，可以使得梯度方向不變的維度上速度變快，梯度方向有所改變的維度上的更新速度變慢，這樣就可以加快收斂並減小震盪。

*   缺點：
    這種情況相當於小球從山上滾下來時是在盲目地沿著坡滾，如果它能具備一些先知，例如快要上坡時，就知道需要減速了的話，適應性會更好。

*   超參數$ \gamma $一般取值為0.9左右

<img src="https://images2018.cnblogs.com/blog/1192699/201803/1192699-20180310220823806-1122734332.png" width="50%" height="50%">

### Nesterov Accelerated Gradient
*   梯度跟新規則:\
    用 $\theta−\gamma v_{t−1} $ 來近似當做引數下一步會變成的值，則在計算梯度時，不是在當前位置，而是未來的位置上\
    $v_t = \gamma v_{t-1} + \eta \nabla_\theta J(\theta - \gamma v_{t-1}) $\
    $\theta = \theta - v_t$


*   超參數$\gamma$一般取值為0.9左右

*   效果比較
    藍色為Momentum，會先計算當前的梯度，然後在更新後的累積梯度後會有一個大的跳躍。\
    NAG 會先在前一步的累積梯度上(brown vector)有一個大的跳躍，然後衡量一下梯度做一下修正(red vector)，這種預期的更新可以避免我們走的太快。\
    NAG 可以使 RNN 在很多地方上有更好的表現。
    
<img src="https://images2018.cnblogs.com/blog/1192699/201803/1192699-20180310224024153-1974893457.png" width="50%" height="50%">

### Adagrad （Adaptive gradient algorithm）
這個演算法就可以對低頻的引數做較大的更新，對高頻的做較小的更新，也因此，對於稀疏的資料它的表現很好，很好地提高了 SGD 的穩健性，例如識別 Youtube 視訊裡面的貓，訓練 GloVe word embeddings，因為它們都是需要在低頻的特徵上有更大的更新。

*   梯度更新規則:

    $\theta _{t+1,i} = \theta_{t,i} - \frac{\eta}{\sqrt{G_{t,ii}+\epsilon}} \cdot g_{t,i}$

    其中，g為 : t時刻參數$\theta_{i}$的梯度

    $g_{t,i} = \nabla_\theta J(\theta_{i})$

    例如,
    普通的SGD，$\theta_{i}$在每一時刻的梯度更新公式為:

    $\theta_{t+1,i} = \theta_{t,i} - \eta \cdot g_{t,i}$

    但這裡的 learning rate $\eta$ 會隨著t和i變:

    $\theta _{t+1,i} = \theta_{t,i} - \frac{\eta}{\sqrt{G_{t,ii}+\epsilon}} \cdot g_{t,i}$

    其中，$G_t$是對角矩陣，(i,i)元素是t時刻參數 $\theta_i$ 的梯度平方和

*   優點\
    減少了學習率的手動調節


*   缺點\
    分母會不斷積累，這樣學習率就會收縮並最終會變得非常小。

超引數設定值：一般η選取0.01

### Adadelta
*   梯度更新規則 : \
    這個演算法是對 Adagrad 的改進，和 Adagrad 相比，就是分母的 G 換成了過去的梯度平方的衰減平均值，指數衰減平均值
    
    $\Delta \theta_t = -\frac{\eta}{\sqrt{E[g^2]_t + \epsilon}} g_t $

    這個分母相當於梯度的均方根 root mean squared (RMS)，在資料統計分析中，將所有值平方求和，求其均值，再開平方，就得到均方根值 ，所以可以用 RMS 簡寫：

    $\Delta \theta_t = -\frac{\eta}{RMS[g]_t} g_t $

    其中 E 的計算公式如下，t 時刻的依賴於前一時刻的平均和當前的梯度：

    $E[g^2]_t = \gamma E[g^2]_{t-1} + (1-\gamma)g^2_t$

    此外，還將學習率 $\eta$ 換成了 $RMS[\Delta \theta]$，這樣的話，我們甚至都不需要提前設定學習率了：

    $\Delta \theta_t = -\frac{RMS[\Delta \theta]_{t-1}}{RMS[g]_t} g_t $
    
    $\theta_{t+1} = \theta_t + \Delta \theta _t$

超引數設定值:  γ 一般設定為 0.9


### RMSprop
RMSprop 是 Geoff Hinton 提出的一種自適應學習率方法。
*   RMSprop 和 Adadelta 都是為了解決 Adagrad 學習率急劇下降問題的，

*   梯度更新規則:\
    RMSprop 與 Adadelta 的第一種形式相同：（使用的是指數加權平均，旨在消除梯度下降中的擺動，與Momentum的效果一樣，某一維度的導數比較大，則指數加權平均就大，某一維度的導數比較小，則其指數加權平均就小，這樣就保證了各維度導數都在一個量級，進而減少了擺動。允許使用一個更大的學習率η）
    
    $E[g^2]_t = 0.9E[g^2]_{t-1} + 0.1 g^2_t$
    
    $\theta_{t+1} = \theta_t - \frac{\eta}{E[g^2]_t + \epsilon} g_t$

超參數設定值 : Hinton 建議設定 $\gamma$ 為 $0.9$， 學習率 $\eta$ 為 $0.001$。

### Adam：Adaptive Moment Estimation
*   這個演算法是另一種計算每個引數的自適應學習率的方法。相當於 RMSprop + Momentum。\
    除了像 Adadelta 和 RMSprop 一樣儲存了過去梯度的平方 $v_t$ 的指數衰減平均值 ，也像 momentum 一樣保持了過去梯度 $m_t$ 的指數衰減平均值：

    $m_t = \beta_1m_{t-1} + (1-\beta_1)g_t$

    $v_t = \beta_2v_{t-1} + (1-\beta_2)g^2_t$

    如果 $m_t$ 和 $v_t$ 被初始化為 0 向量，那它們就會向 0 偏置，所以做了偏差校正，通過計算偏差校正後的 mt 和 vt 來抵消這些偏差：

    $\hat{m}_t = \frac{m_t}{1-\beta^t_1}$

    $\hat{v}_t = \frac{v_t}{1-\beta^t_2}$

*   梯度更新規則:

    $\theta = \theta - \frac{\eta}{\sqrt{\hat{v}_t} + \epsilon} \hat{m}_t$

*   超參數設定值    :   建議 $\beta_1 ＝ 0.9，\beta_2 ＝ 0.999，\epsilon ＝ 1e−8$

*   實踐表明，Adam 比其他適應性學習方法效果要好。

### 不同演算法在鞍點和等高線上的表現：

SGD optimization on saddle point

<img src="https://images2018.cnblogs.com/blog/1192699/201803/1192699-20180311105558593-251578131.gif" width="50%" height="50%">

SGD optimization on loss surface contours

<img src="https://images2018.cnblogs.com/blog/1192699/201803/1192699-20180311110108768-2113908893.gif" width="50%" height="50%">

上面兩種情況都可以看出，Adagrad, Adadelta, RMSprop 幾乎很快就找到了正確的方向並前進，收斂速度也相當快，而其它方法要麼很慢，要麼走了很多彎路才找到。

由圖可知自適應學習率方法即 Adagrad, Adadelta, RMSprop, Adam 在這種情景下會更合適而且收斂性更好。

### 如何選擇優化演算法
如果資料是稀疏的，就用自適用方法，即 Adagrad, Adadelta, RMSprop, Adam。

RMSprop, Adadelta, Adam 在很多情況下的效果是相似的。

Adam 就是在 RMSprop 的基礎上加了 bias-correction 和 momentum，

隨著梯度變的稀疏，Adam 比 RMSprop 效果會好。

整體來講，Adam 是最好的選擇。

很多論文裡都會用 SGD，沒有 momentum 等。SGD 雖然能達到極小值，但是比其它演算法用的時間長，而且可能會被困在鞍點。

如果需要更快的收斂，或者是訓練更深更復雜的神經網路，需要用一種自適應的演算法。